# **Base Evaluation**

## **Config**

In [2]:
import os

seed = 42
gpu_ids = "7"

METRICS_FILE   = "evaluation_metrics.csv"
DATASET_PATH   = "../datasets/cqa_test_prompt_completion.jsonl"
BATCH_SIZE     = 128

MODEL_NAMES = {
    "Meta-Llama-3.1-8B": "../model_cache/Meta-Llama-3.1-8B",
    "Aya-23-8B":         "../model_cache/Aya-23-8B",
    "SeaLLMs-v3-7B":     "../model_cache/SeaLLMs-v3-7B",
    "SEA-LION-v3-8B":    "../model_cache/SEA-LION-v3-8B",
    "Sahabat-AI-8B":     "../model_cache/Sahabat-AI-8B"
}

## **Import Libraries**

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_ids

import gc
import time
import signal
import random

import torch
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    set_seed
)
import evaluate

pd.set_option("display.max_colwidth", None)

/raid/home/llmsosmed/rlaif/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-06-07 02:17:11,199] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


## **Utility Functions**

In [4]:
def set_global_seed(s: int = seed):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark     = False
    set_seed(s)

def load_model_and_tokenizer(model_path: str, use_auth_token: str = None):
    set_global_seed()
    tokenizer = AutoTokenizer.from_pretrained(
        model_path,
        local_files_only=True,
        token=use_auth_token
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        quantization_config=quant_config,
        local_files_only=True,
        token=use_auth_token
    )
    model.eval()
    return tokenizer, model

def generate_batch(tokenizer, model, prompts, max_new_tokens=256):
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1792
    ).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return [
        text[len(prompt):]#.split("\n", 1)[0]
        for text, prompt in zip(decoded, prompts)
    ]

In [5]:
set_global_seed()

## **Load Dataset**

In [6]:
df = pd.read_json(DATASET_PATH, lines=True)
print(f"Total examples: {len(df)}")
df.sample(3)

Total examples: 340


,prompt,completion
284,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 5 (1) Kontrak Investasi Kolektif DIRE Syariah, selain wajib memenuhi ketentuan yang diatur dalam peraturan perundang-undangan di sektor Pasar Modal yang mengatur mengenai Dana Investasi Real Estat berbentuk Kontrak Investasi Kolektif sebagaimana dimaksud dalam Pasal 4, wajib pula memuat ketentuan paling sedikit: a. Manajer Investasi dan Bank Kustodian merupakan wakil (wakiliin) yang bertindak untuk kepentingan para pemegang Unit Penyertaan DIRE Syariah berbentuk Kontrak Investasi Kolektif sebagai pihak yang diwakili (muwakil) dimana Manajer Investasi diberi wewenang untuk mengelola portofolio investasi kolektif dan Bank Kustodian diberi wewenang untuk melaksanakan Penitipan Kolektif; b. aset Real Estat, Aset Yang Berkaitan Dengan Real Estat, dan/atau kas dan setara kas tidak bertentangan dengan Prinsip Syariah di Pasar Modal; c. mekanisme pembersihan aset Real Estat, Aset Yang Berkaitan Dengan Real Estat, dan/atau kas dan setara kas dari unsur yang bertentangan dengan\n\nQuestion: Apa saja ketentuan minimal yang harus dimuat dalam Kontrak Investasi Kolektif DIRE Syariah?\nAnswer:","Mekanisme proses dijelaskan sebagai Manajer Investasi dan Bank Kustodian sebagai wakil, aset tidak bertentangan dengan Prinsip Syariah, mekanisme pembersihan aset, kata 'Syariah' pada nama, akad syariah dan skema transaksi, ringkasan akad, besarnya nisbah pembayaran, dan rencana jadwal serta tata cara pembagian hasil."
116,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\n6. \nPeminjam adalah nasabah perorangan, perusahaan, atau \nbadan yang memperoleh Penyediaan Dana dari Bank, \ntermasuk: \na. \ndebitur, untuk Penyediaan Dana berupa kredit atau \npembiayaan; \nb. \npenerbit surat berharga, pihak yang menjual surat \nberharga, \nmanajer \ninvestasi \nkontrak \ninvestasi \nkolektif, \ndan/atau \nentitas \nreferensi \n(reference entity), untuk Penyediaan Dana berupa \nsurat berharga; \nc. \npihak yang mengalihkan risiko kredit (protection \nbuyer) dan/atau entitas referensi (reference entity), \nuntuk Penyediaan Dana berupa derivatif kredit \n(credit derivatives); \nd. \npemohon (applicant), untuk Penyediaan Dana berupa \njaminan (guarantee), letter of credit, standby letter of \ncredit, atau instrumen serupa lain; \ne. \npihak tempat Bank melakukan penyertaan modal \n(investee), \nuntuk \nPenyediaan \nDana \nberupa \npenyertaan modal; \nf. \nBank atau debitur, untuk Penyediaan Dana berupa \ntagihan akseptasi; \ng. \npihak \nlawan \ntransaksi \n(counterparty), \nuntuk \nPenyediaan Dana berupa penempatan dan transaksi \nderivatif; dan/atau \nh. \npihak lain yang wajib melunasi tagihan kepada Bank.\n\nQuestion: Siapa saja yang termasuk dalam definisi Peminjam menurut dokumen ini?\nAnswer:","Dokumen ini mencakup nasabah perorangan, perusahaan, atau badan yang memperoleh Penyediaan Dana dari Bank sebagai Peminjam."
113,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 3 (1) Setiap Pihak yang melanggar ketentuan sebagaimana dimaksud dalam Pasal 2 ayat (2), dikenakan sanksi administratif. (2) Sanksi sebagaimana dimaksud pada ayat (1) dikenakan juga kepada Pihak yang menyebabkan terjadinya pelanggaran sebagaimana dimaksud pada ayat (1). (3) Sanksi sebagaimana dimaksud pada ayat (1) dan ayat (2) dijatuhkan oleh Otoritas jasa Keuangan. (4) Sanksi administratif sebagaimana dimaksud pada ayat (1) berupa: a. peringatan tertulis; b. denda yaitu kewajiban untuk membayar sejumlah uang tertentu; c. pembatasan kegiatan usaha; d. pembekuan kegiatan usaha; e. pencabutan izin usaha; f. pem

## **Prepare Metrics**

In [7]:
if not os.path.exists(METRICS_FILE):
    pd.DataFrame(columns=[
        "model","exact_match","rouge1_f1","rouge2_f1","rougeL_f1",
        "bleu","meteor","inference_time_sec"
    ]).to_csv(METRICS_FILE, index=False)

# Load evaluators
em    = evaluate.load("exact_match")
rouge = evaluate.load("rouge")
bleu  = evaluate.load("bleu")
meteor= evaluate.load("meteor")

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/llmsosmed/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/llmsosmed/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/llmsosmed/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## **Inference & Evaluation**

In [ ]:
for model_key, model_id in MODEL_NAMES.items():
    print(f"\n▶ Evaluating {model_key}")
    tokenizer, model = load_model_and_tokenizer(model_id, use_auth_token=os.getenv("HF_TOKEN"))

    # —– Cetak statistik bobot LoRA
    print("=== LoRA weights for", model_key, "===")
    for name, param in model.named_parameters():
        if "lora" in name:
            print(name, param.norm().item()) 
    print("="*30)
    
    # Warm-up
    _ = model.generate(
        **tokenizer("Warm up", return_tensors="pt").to(model.device),
        max_new_tokens=1,
        pad_token_id=tokenizer.eos_token_id
    )

    preds, refs, details = [], [], []
    start = time.time()

    for i in tqdm(range(0, len(df), BATCH_SIZE), desc=model_key):
        batch = df.iloc[i : i + BATCH_SIZE]
        prompts = batch["prompt"].tolist()
        batch_preds = generate_batch(tokenizer, model, prompts)

        preds.extend(batch_preds)
        refs.extend(batch["completion"].tolist())

        # Simpan detail
        for prompt, ref, pred in zip(prompts, batch["completion"], batch_preds):
            details.append({
                "prompt": prompt,
                "ground_truth": ref,
                model_key: pred
            })

    inf_time = time.time() - start
    print(f"→ Inference time: {inf_time:.1f} sec")

    # Compute metrics
    r_em  = em.compute(predictions=preds, references=refs)["exact_match"]
    r_rog = rouge.compute(predictions=preds, references=refs)
    r_ble = bleu.compute(predictions=preds, references=[[r] for r in refs])["bleu"]
    r_met = meteor.compute(predictions=preds, references=refs)["meteor"]

    # Append to CSV
    row = {
        "model":             model_key,
        "exact_match":       r_em,
        "rouge1_f1":         r_rog["rouge1"],
        "rouge2_f1":         r_rog["rouge2"],
        "rougeL_f1":         r_rog["rougeL"],
        "bleu":              r_ble,
        "meteor":            r_met,
        "inference_time_sec": inf_time
    }
    pd.DataFrame([row]).to_csv(METRICS_FILE, mode="a", header=False, index=False)

    # Save detailed predictions
    pd.DataFrame(details).to_json(
        f"detailed_{model_key}.jsonl",
        orient="records",
        lines=True
    )

    # Cleanup
    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()


▶ Evaluating Meta-Llama-3.1-8B


Loading checkpoint shards: 100%|██████████| 4/4 [00:14<00:00,  3.64s/it]


=== LoRA weights for Meta-Llama-3.1-8B ===


Meta-Llama-3.1-8B:   0%|          | 0/3 [00:00<?, ?it/s]/raid/home/llmsosmed/rlaif/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/raid/home/llmsosmed/rlaif/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Meta-Llama-3.1-8B: 100%|██████████| 3/3 [04:04<00:00, 81.60s/it]


→ Inference time: 244.8 sec

▶ Evaluating Aya-23-8B


...

SeaLLMs-v3-7B: 100%|██████████| 3/3 [03:11<00:00, 63.79s/it]


→ Inference time: 191.4 sec

▶ Evaluating SEA-LION-v3-8B


Loading checkpoint shards: 100%|██████████| 4/4 [00:14<00:00,  3.64s/it]


=== LoRA weights for SEA-LION-v3-8B ===


SEA-LION-v3-8B:  33%|███▎      | 1/3 [01:29<02:59, 89.76s/it]

## **Results**

In [ ]:
dfm = pd.read_csv(METRICS_FILE)
print(dfm.to_markdown(index=False))

In [ ]:
merged = None

for key in MODEL_NAMES:
    path = f"detailed_{key}.jsonl"
    if not os.path.exists(path):
        continue

    tmp = (
        pd.read_json(path, lines=True)
          .rename(columns={key: f"pred_{key}"})
          [["prompt", "ground_truth", f"pred_{key}"]]
    )

    if merged is None:
        merged = tmp
    else:
        merged = merged.merge(
            tmp,
            on=["prompt", "ground_truth"],
            how="outer"
        )

# Tampilkan hasil
print(f"Total rows in merged DataFrame: {len(merged)}")
display(merged.head(5))

In [ ]:
import signal

os.kill(os.getpid(), signal.SIGTERM)